# 🎓 學術主頁生成器 v2 - 多模板版

**AI 解析 + 4 種模板風格可選 + GitHub Pages 即用**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/xhh678876/personal_page/blob/main/Homepage_Generator_v2_Colab.ipynb)

---

## ✨ v2 新功能

- 🎨 **4 種模板風格** - 生成前可選擇
- 🌙 深色科技風 - 深藍背景、科技感
- 🌈 紫色漸變風 - 紫色漸變、玻璃態
- 📖 極簡學術風 - 白底經典排版
- 💚 霓虹賽博風 - 黑底霓虹極客
- 📦 **GitHub Pages 即用** - 下載 ZIP 直接部署

---

## 📝 使用步驟

1. **運行所有單元** - Runtime → Run all
2. **點擊公網鏈接** - `https://xxx.gradio.live`
3. **設置** - 選擇 AI + 輸入 Key + 上傳 PDF
4. **選擇模板** - 選擇喜歡的風格
5. **生成** - 下載 ZIP 部署到 GitHub

## 步驟 1: 安裝依賴

In [ ]:
!pip install -q gradio google-generativeai openai pdf2image pillow
!apt-get update -qq && apt-get install -y -qq poppler-utils
print("✅ 依賴安裝完成！")

## 步驟 2: 啟動 v2 應用

In [ ]:
import gradio as gr
import google.generativeai as genai
from openai import OpenAI
import base64, json, io, os, zipfile, shutil
from pdf2image import convert_from_path
from PIL import Image

# ============ 模板 1: 深色科技風 ============
def template_dark_tech(data):
    sections_html = ""
    for section in data.get("sections", []):
        items_html = "".join([f'<div class="item"><h3>{i.get("title","")}</h3><p class="sub">{i.get("subtitle","")}</p><p class="date">{i.get("date","")}</p>{f"<p class=desc>{i['description']}</p>" if i.get("description") else ""}</div>' for i in section.get("items",[])])
        sections_html += f'<section><h2>{section.get("title","")}</h2>{items_html}</section>'
    return f'''<!DOCTYPE html><html><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width,initial-scale=1"><title>{data.get('name','')}</title>
<style>*{{margin:0;padding:0;box-sizing:border-box}}body{{font-family:-apple-system,sans-serif;background:linear-gradient(135deg,#0f172a,#1e293b);color:#f8fafc;min-height:100vh;padding:2rem}}.container{{max-width:1000px;margin:0 auto}}.hero{{text-align:center;padding:4rem 2rem}}h1{{font-size:3.5rem;font-weight:800;background:linear-gradient(135deg,#667eea,#00d4ff);-webkit-background-clip:text;-webkit-text-fill-color:transparent;margin-bottom:1rem}}.title{{font-size:1.3rem;color:#00d4ff;margin-bottom:2rem}}.bio{{max-width:700px;margin:0 auto 2rem;line-height:1.8;opacity:0.9}}.contact a{{color:#00d4ff;margin:0 1rem;text-decoration:none;padding:0.5rem 1.5rem;border:1px solid #00d4ff;border-radius:25px;transition:all 0.3s}}.contact a:hover{{background:#00d4ff;color:#0f172a}}section{{margin:3rem 0;padding:2rem;background:rgba(255,255,255,0.05);border-radius:12px;border-left:3px solid #667eea}}h2{{color:#667eea;margin-bottom:1.5rem;font-size:1.8rem}}.item{{margin-bottom:2rem;padding-left:1rem;border-left:2px solid #00d4ff}}h3{{color:#00d4ff;margin-bottom:0.3rem}}.sub{{color:#94a3b8;margin-bottom:0.2rem}}.date{{color:#64748b;font-size:0.9rem;margin-bottom:0.5rem}}.desc{{color:#cbd5e1;line-height:1.6;white-space:pre-wrap}}</style></head>
<body><div class="container"><div class="hero"><h1>{data.get('name','')}</h1><p class="title">{data.get('title','')}</p><p class="bio">{data.get('bio','')}</p><div class="contact">{f'<a href="mailto:{data["email"]}">📧 Email</a>' if data.get('email') else ''}{f'<a href="https://{data["website"]}" target="_blank">🌐 Website</a>' if data.get('website') else ''}</div></div>{sections_html}</div></body></html>'''

# ============ 模板 2: 紫色漸變風 ============
def template_gradient_purple(data):
    sections_html = ""
    for section in data.get("sections", []):
        items_html = "".join([f'<div class="card"><h3>{i.get("title","")}</h3><p class="sub">{i.get("subtitle","")}</p><span class="date">{i.get("date","")}</span>{f"<p class=desc>{i['description']}</p>" if i.get("description") else ""}</div>' for i in section.get("items",[])])
        sections_html += f'<section><h2>{section.get("title","")}</h2><div class="grid">{items_html}</div></section>'
    return f'''<!DOCTYPE html><html><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width,initial-scale=1"><title>{data.get('name','')}</title>
<style>*{{margin:0;padding:0;box-sizing:border-box}}body{{font-family:-apple-system,sans-serif;background:linear-gradient(135deg,#667eea,#764ba2);min-height:100vh;padding:2rem}}.container{{max-width:1200px;margin:0 auto;background:rgba(255,255,255,0.95);border-radius:20px;padding:3rem;box-shadow:0 20px 60px rgba(0,0,0,0.3)}}.hero{{text-align:center;padding:2rem 0 3rem;border-bottom:2px solid #eee;margin-bottom:2rem}}h1{{font-size:3rem;font-weight:800;background:linear-gradient(135deg,#667eea,#764ba2);-webkit-background-clip:text;-webkit-text-fill-color:transparent}}.title{{color:#666;font-size:1.2rem;margin:1rem 0}}.bio{{color:#444;max-width:700px;margin:1rem auto;line-height:1.8}}.contact a{{color:#667eea;margin:0 0.5rem;text-decoration:none}}section{{margin:2rem 0}}h2{{color:#333;font-size:1.8rem;margin-bottom:1rem;padding-bottom:0.5rem;border-bottom:3px solid #667eea}}.grid{{display:grid;grid-template-columns:repeat(auto-fill,minmax(300px,1fr));gap:1.5rem}}.card{{background:#f8f9fa;padding:1.5rem;border-radius:12px;border-left:4px solid #667eea;transition:all 0.3s}}.card:hover{{transform:translateY(-5px);box-shadow:0 10px 30px rgba(0,0,0,0.1)}}h3{{color:#333;margin-bottom:0.5rem}}.sub{{color:#666}}.date{{color:#999;font-size:0.9rem}}.desc{{color:#555;line-height:1.6;margin-top:0.5rem;white-space:pre-wrap}}</style></head>
<body><div class="container"><div class="hero"><h1>{data.get('name','')}</h1><p class="title">{data.get('title','')}</p><p class="bio">{data.get('bio','')}</p><div class="contact">{f'<a href="mailto:{data["email"]}">📧 {data["email"]}</a>' if data.get('email') else ''}{f'<a href="https://{data["website"]}" target="_blank">🌐 {data["website"]}</a>' if data.get('website') else ''}</div></div>{sections_html}</div></body></html>'''

# ============ 模板 3: 極簡學術風 ============
def template_academic_minimal(data):
    sections_html = ""
    for section in data.get("sections", []):
        items_html = "".join([f'<div class="entry"><div class="meta"><span class="date">{i.get("date","")}</span></div><div class="content"><h3>{i.get("title","")}</h3><p class="org">{i.get("subtitle","")}</p>{f"<p class=desc>{i['description']}</p>" if i.get("description") else ""}</div></div>' for i in section.get("items",[])])
        sections_html += f'<section><h2>{section.get("title","")}</h2>{items_html}</section>'
    return f'''<!DOCTYPE html><html><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width,initial-scale=1"><title>{data.get('name','')}</title>
<style>*{{margin:0;padding:0;box-sizing:border-box}}body{{font-family:Georgia,'Times New Roman',serif;background:#fafafa;color:#333;line-height:1.6;padding:2rem}}.container{{max-width:800px;margin:0 auto;background:white;padding:4rem;box-shadow:0 2px 10px rgba(0,0,0,0.1)}}h1{{font-size:2.5rem;color:#2c3e50;margin-bottom:0.5rem;font-weight:600}}.title{{color:#7f8c8d;font-size:1.1rem;margin-bottom:1rem}}.bio{{color:#555;margin:2rem 0;font-size:1.05rem}}.contact{{margin-bottom:3rem;padding-bottom:2rem;border-bottom:2px solid #e0e0e0}}.contact a{{color:#3498db;text-decoration:none;margin-right:1.5rem}}section{{margin-bottom:2.5rem}}h2{{font-size:1.5rem;color:#2c3e50;margin-bottom:1rem;padding-bottom:0.5rem;border-bottom:1px solid #bdc3c7}}.entry{{display:flex;margin-bottom:1.5rem}}.meta{{width:120px;flex-shrink:0}}.date{{color:#7f8c8d;font-size:0.9rem}}.content{{flex:1}}h3{{font-size:1.1rem;color:#34495e;font-weight:600;margin-bottom:0.3rem}}.org{{color:#7f8c8d;font-size:0.95rem}}.desc{{color:#555;margin-top:0.5rem;white-space:pre-wrap}}@media(max-width:600px){{.entry{{flex-direction:column}}.meta{{width:100%;margin-bottom:0.3rem}}}}</style></head>
<body><div class="container"><h1>{data.get('name','')}</h1><p class="title">{data.get('title','')}</p><div class="contact">{f'<a href="mailto:{data["email"]}">{data["email"]}</a>' if data.get('email') else ''}{f'<a href="https://{data["website"]}" target="_blank">{data["website"]}</a>' if data.get('website') else ''}</div>{f'<p class="bio">{data["bio"]}</p>' if data.get('bio') else ''}{sections_html}</div></body></html>'''

# ============ 模板 4: 霓虹賽博風 ============
def template_neon_cyber(data):
    sections_html = ""
    for section in data.get("sections", []):
        items_html = "".join([f'<div class="item"><h3>{i.get("title","")}</h3><p class="sub">{i.get("subtitle","")}</p><p class="date">{i.get("date","")}</p>{f"<p class=desc>{i['description']}</p>" if i.get("description") else ""}</div>' for i in section.get("items",[])])
        sections_html += f'<section><h2>&gt; {section.get("title","")}</h2>{items_html}</section>'
    return f'''<!DOCTYPE html><html><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width,initial-scale=1"><title>{data.get('name','')}</title>
<style>*{{margin:0;padding:0;box-sizing:border-box}}body{{font-family:'Courier New',monospace;background:#0a0a0a;color:#e0e0e0;padding:2rem;min-height:100vh}}.container{{max-width:900px;margin:0 auto;background:rgba(20,20,20,0.9);border:1px solid #00ff41;border-radius:10px;padding:3rem;box-shadow:0 0 50px rgba(0,255,65,0.1)}}h1{{font-size:3rem;color:#00ff41;text-transform:uppercase;letter-spacing:0.2rem;margin-bottom:1rem;text-shadow:0 0 10px rgba(0,255,65,0.5)}}.title{{color:#00d4ff;margin-bottom:1rem;font-size:1.2rem}}.bio{{color:#ccc;line-height:1.8;margin-bottom:2rem}}.contact{{margin-bottom:3rem}}.contact a{{color:#00ff41;text-decoration:none;padding:0.5rem 1rem;border:1px solid #00ff41;border-radius:5px;margin-right:1rem;transition:all 0.3s}}.contact a:hover{{background:#00ff41;color:#000;box-shadow:0 0 20px rgba(0,255,65,0.6)}}section{{margin-bottom:3rem;padding-bottom:2rem;border-bottom:1px solid #333}}h2{{color:#00d4ff;font-size:1.8rem;margin-bottom:1.5rem;text-transform:uppercase;letter-spacing:0.1rem}}.item{{margin-bottom:2rem;padding-left:1.5rem;border-left:2px solid #00ff41}}h3{{color:#fff;margin-bottom:0.5rem}}.sub{{color:#00d4ff;margin-bottom:0.3rem}}.date{{color:#888;font-size:0.9rem;margin-bottom:0.5rem}}.desc{{color:#ccc;line-height:1.6;white-space:pre-wrap}}</style></head>
<body><div class="container"><h1>{data.get('name','')}</h1><p class="title">{data.get('title','')}</p><p class="bio">{data.get('bio','')}</p><div class="contact">{f'<a href="mailto:{data["email"]}">EMAIL</a>' if data.get('email') else ''}{f'<a href="https://{data["website"]}" target="_blank">WEBSITE</a>' if data.get('website') else ''}</div>{sections_html}</div></body></html>'''

TEMPLATES = {
    "🌙 深色科技風": template_dark_tech,
    "🌈 紫色漸變風": template_gradient_purple,
    "📖 極簡學術風": template_academic_minimal,
    "💚 霓虹賽博風": template_neon_cyber,
}

def pdf_to_images(pdf_file):
    return convert_from_path(pdf_file, dpi=150)

def parse_with_gemini(images, api_key):
    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-2.0-flash-exp')
        prompt = '分析學術簡歷，返回JSON：{"name":"","title":"","email":"","website":"","bio":"","sections":[{"title":"","items":[{"title":"","subtitle":"","date":"","description":""}]}]}'
        response = model.generate_content([prompt] + images)
        text = response.text.strip()
        if text.startswith("```"): text = text.split("```")[1].replace("json","").strip()
        return json.loads(text), None
    except Exception as e:
        return None, str(e)

def parse_with_openai(images, api_key):
    try:
        client = OpenAI(api_key=api_key)
        contents = [{"type":"image_url","image_url":{"url":f"data:image/png;base64,{base64.b64encode(io.BytesIO(img._repr_png_()).getvalue() if hasattr(img,'_repr_png_') else (lambda b: (img.save(b,format='PNG'),b.getvalue())[1])(io.BytesIO())).decode()}"}} for img in images]
        response = client.chat.completions.create(model="gpt-4o",messages=[{"role":"user","content":[{"type":"text","text":"分析學術簡歷返回JSON"},*contents]}],max_tokens=4000)
        text = response.choices[0].message.content.strip()
        if text.startswith("```"): text = text.split("```")[1].replace("json","").strip()
        return json.loads(text), None
    except Exception as e:
        return None, str(e)

def generate_project(data, template_func):
    output_dir = "homepage_project"
    if os.path.exists(output_dir): shutil.rmtree(output_dir)
    os.makedirs(output_dir)
    html = template_func(data)
    with open(os.path.join(output_dir,"index.html"),"w",encoding="utf-8") as f: f.write(html)
    with open(os.path.join(output_dir,"README.md"),"w",encoding="utf-8") as f: f.write(f"# {data.get('name','')} Homepage\n\n上傳到GitHub→Settings→Pages→完成")
    zip_path = f"{output_dir}.zip"
    with zipfile.ZipFile(zip_path,'w') as z:
        for r,d,fs in os.walk(output_dir):
            for f in fs: z.write(os.path.join(r,f),os.path.relpath(os.path.join(r,f),output_dir))
    return html, zip_path

def process(pdf, provider, key, template, progress=gr.Progress()):
    if not pdf: return None,None,None,"❌ 請上傳PDF"
    if not key: return None,None,None,"❌ 請輸入Key"
    try:
        progress(0.2,desc="轉換PDF...")
        images = pdf_to_images(pdf)
        progress(0.4,desc=f"用{provider}解析...")
        data,err = parse_with_gemini(images,key) if provider=="Gemini" else parse_with_openai(images,key)
        if err: return None,None,None,f"❌ {err}"
        progress(0.7,desc=f"生成{template}...")
        html,zip_path = generate_project(data, TEMPLATES[template])
        return html, zip_path, json.dumps(data,ensure_ascii=False,indent=2), f"✅ 成功！{template}"
    except Exception as e:
        return None,None,None,f"❌ {e}"

with gr.Blocks(title="學術主頁生成器v2",theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🎓 學術主頁生成器 v2\n> 4種模板風格可選")
    with gr.Row():
        with gr.Column(scale=1):
            provider = gr.Radio(["Gemini","OpenAI"],value="Gemini",label="AI")
            key = gr.Textbox(label="API Key",type="password")
            pdf = gr.File(label="簡歷PDF",file_types=[".pdf"],type="filepath")
            template = gr.Radio(list(TEMPLATES.keys()),value="🌙 深色科技風",label="🎨 選擇模板")
            btn = gr.Button("✨ 生成",variant="primary")
            status = gr.Textbox(label="狀態")
        with gr.Column(scale=2):
            with gr.Tab("預覽"): preview = gr.HTML()
            with gr.Tab("下載"): zipf = gr.File(label="ZIP")
            with gr.Tab("JSON"): jsonout = gr.Code(language="json")
    btn.click(process,[pdf,provider,key,template],[preview,zipf,jsonout,status])
    gr.Markdown("---\n**模板**：🌙深色科技 | 🌈紫色漸變 | 📖極簡學術 | 💚霓虹賽博")

demo.launch(share=True,debug=True)

## 🎉 完成！

點擊公網鏈接，選擇喜歡的模板風格，生成後下載 ZIP 部署到 GitHub Pages！

---

**GitHub**: https://github.com/xhh678876/personal_page